In [1]:
prompt1_template = """
You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.
Your task is to analyze the relevance of the generated answer compared to the original answer provided.
Based on the relevance and similarity of the generated answer to the original answer, you will classify
it as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Original Answer: {answer_orig}
Generated Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the original
answer and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

prompt2_template = """
You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [2]:
import pandas as pd
df_gpt4o_mini = pd.read_csv('data/results-gpt4o-mini-crypto.csv')

In [3]:
df_sample = df_gpt4o_mini.sample(n=150, random_state=1)

In [4]:
samples = df_sample.to_dict(orient='records')

In [5]:
record = samples[0]
record

{'answer_llm': 'Ethereum smart contracts can be coded in the following programming languages: Solidity, Serpent, LLL, and Mutan. Additionally, the Ethereum protocol itself has been developed using a variety of languages, including C++, Python, Ruby, Go, Java, and Rust.',
 'answer_orig': 'Like the Bitcoin blockchain, Ethereum utilizes nodes that are operated voluntarily in order to verify transactions in the network. Nodes can contain the entirety or a segment of Ethereum transaction history, the most recent information about the state of smart contracts, the balances of accounts, and more. At the foundation of Ethereum is the Ethereum Virtual Machine (EVM), which is the executable and trustless environment for smart contracts: computer protocols that facilitate, verify, and enforce the negotiation and performance of some sort of digital agreement. The EVM executes a contract with whatever rules the developer initially programmed, such as sending money from Alice to Bob. The EVM execute

In [6]:
prompt = prompt1_template.format(**record)
print(prompt)

You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.
Your task is to analyze the relevance of the generated answer compared to the original answer provided.
Based on the relevance and similarity of the generated answer to the original answer, you will classify
it as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Original Answer: Like the Bitcoin blockchain, Ethereum utilizes nodes that are operated voluntarily in order to verify transactions in the network. Nodes can contain the entirety or a segment of Ethereum transaction history, the most recent information about the state of smart contracts, the balances of accounts, and more. At the foundation of Ethereum is the Ethereum Virtual Machine (EVM), which is the executable and trustless environment for smart contracts: computer protocols that facilitate, verify, and enforce the negotiation and performance of some sort of digital agreement. The EVM executes a contract with w

In [8]:
from openai import OpenAI

client = OpenAI()

def llm(prompt, model='gpt-4o'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [9]:
answer = llm(prompt, model='gpt-4o-mini')

In [10]:
import json

In [12]:
from tqdm.auto import tqdm
evaluations = []

for record in tqdm(samples):
    prompt = prompt1_template.format(**record)
    evaluation = llm(prompt, model='gpt-4o-mini')
    evaluations.append(evaluation)

  0%|          | 0/150 [00:00<?, ?it/s]

In [13]:
json_evaluations = []

for i, str_eval in enumerate(evaluations):
    json_eval = json.loads(str_eval)
    json_evaluations.append(json_eval)

In [14]:
df_evaluations = pd.DataFrame(json_evaluations)

In [15]:
df_evaluations.Relevance.value_counts()

Relevance
RELEVANT           135
PARTLY_RELEVANT     13
NON_RELEVANT         2
Name: count, dtype: int64

In [16]:
df_evaluations[df_evaluations.Relevance == 'NON_RELEVANT'] #.to_dict(orient='records')

,Relevance,Explanation
50,NON_RELEVANT,The generated answer does not address the ques...
77,NON_RELEVANT,The generated answer does not address the spec...


In [17]:
prompt = prompt2_template.format(**record)
print(prompt)

You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: Which blockchain development does the Consensys Academy focus on?
Generated Answer: The Consensys Academy focuses on Ethereum blockchain development.

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}


In [18]:
evaluation = llm(prompt, model='gpt-4o-mini')
print(evaluation)

{
  "Relevance": "RELEVANT",
  "Explanation": "The generated answer directly addresses the question by stating that the Consensys Academy focuses on Ethereum blockchain development, which is the specific topic being inquired about."
}


In [19]:
evaluations_2 = []

for record in tqdm(samples):
    prompt = prompt2_template.format(**record)
    evaluation = llm(prompt, model='gpt-4o-mini')
    evaluations_2.append(evaluation)

  0%|          | 0/150 [00:00<?, ?it/s]

In [20]:
json_evaluations_2 = []

for i, str_eval in enumerate(evaluations_2):
    json_eval = json.loads(str_eval)
    json_evaluations_2.append(json_eval)

In [21]:
df_evaluations_2 = pd.DataFrame(json_evaluations_2)

In [22]:
df_evaluations_2[df_evaluations_2.Relevance == 'NON_RELEVANT']

,Relevance,Explanation
50,NON_RELEVANT,The generated answer states that it cannot pro...
77,NON_RELEVANT,The generated answer states that the context d...


In [23]:
df_evaluations.to_csv('data/evaluations-aqa.csv', index=False)
df_evaluations_2.to_csv('data/evaluations-qa.csv', index=False)